**ORC ver -- 1.2.0**

In [2]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
import cv2
from PIL import Image
import os

**GETTING THE MINIST DATA**

In [3]:
def get_mnist_data(): 
    path = 'mnist.npz'
    mnist =tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data(path=path)
    return x_train, y_train, x_test, y_test



**TRAINING THE MODEL**

In [4]:
def train_model(x_train, y_train, x_test, y_test): 
    #stop the training at the 99 percent accuracy
    class MyCallback(tf.keras.callbacks.Callback): 
        def on_epoch_end(self, epoch, logs=None):
            if logs.get('accuracy')>0.99: 
                print("\nReached 99% accuracy so cancelling training!")
                self.model.stop_training = True
    callbacks = MyCallback()

    #Normalizing Data
    x_train, x_test = x_train/255.0, x_test/255.0
    #NOW WE ARE GOING TO CREATE THE FUCKING MODEL 
    model = tf.keras.models.Sequential([

        tf.keras.layers.Flatten(input_shape = (28,28)),
        tf.keras.layers.Dense(512, activation= 'relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer= 'adam', 
                  loss= 'sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    #NOW WE ARE GOING TO TRAIN THE MODEL
    model.fit(x_train, y_train, epochs = 10,callbacks=[callbacks])
    return model
#Loading the fucking DATA
(x_train, y_train, x_test, y_test)= get_mnist_data()

#Trining the model
model = train_model(x_train, y_train,x_test, y_test )
print("MODEL TRAINED SUCESSFULLY!!!!")


c:\Users\muzam\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9038 - loss: 0.3379
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9743 - loss: 0.0811
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9842 - loss: 0.0504
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9889 - loss: 0.0346
Epoch 5/10
1858/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9919 - loss: 0.0250
Reached 99% accuracy so cancelling training!
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9919 - loss: 0.0250
MODEL TRAINED SUCESSFULLY!!!!


**SAVING AND LOADING THE MODEL**

In [5]:
model.save('model.keras')
print('MODEL SAVED!!!')

#LOADING THE MODEL
model_loded = tf.keras.models.load_model('model.keras')
print("MODEL IS LOADED SUCESS FULLY!!!!")




MODEL SAVED!!!
MODEL IS LOADED SUCESS FULLY!!!!


**Predict Digits Using the Model**

In [6]:
def predict(model,img): 
    imgs= np.array([img])
    res = model.predict(imgs)
    return str(np.array(res))



**LOADING THE TEST IMAGES**


i have created images on paint in 28x28 format


In [7]:
def load_custom_images(image_folder):
    images = []
    labels = []
    for filename in os.listdir(image_folder): 
        if filename.endswith('.png') or filename.endswith('.jpg'): 
            label=int (filename.split('.')[0])
            img_path = os.path.join(image_folder,filename)
            img = Image.open(img_path).convert('L') #Convert to the grey scale
            img = img.resize((28,28), Image.LANCZOS) #Resize the image
            img = np.array(img)
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

**PREDICT THE IMAGES USING THE MODEL WE HAVE TRAINED**|

In [8]:
def predict(model, img): 
    img= np.expand_dims(img, axis = 0 )
    img = img/225.0 # as i have told that this normalizing the image because the limit of the image in the greyscale is 0 to 225
    pred = model.predict(img)
    return np.argmax(pred)


**THE MAIN FUNCTION**

In [11]:
image_number = 1
while True:
    image_path = rf'C:\Users\muzam\OneDrive\Desktop\OCR\digits\{image_number}.png'  # Use raw string to handle backslashes
    
    if not os.path.isfile(image_path):
        print('No more images found.')
        break
    
    try:
        # Open and process the image
        image = Image.open(image_path).convert('L')  # Convert to grayscale
        image = np.array(image)  # Convert to NumPy array

        # Check the shape of the image and preprocess accordingly
        if image.shape != (28, 28):
            image = np.resize(image, (28, 28))  # Resize if necessary (optional)

        image = np.invert(image)  # Invert the colors
        image = image.reshape(28, 28, 1)  # Ensure it matches the expected input shape for the model

        # Predict and print result
        prediction = model.predict(image.reshape(1, 28, 28, 1))  # Adjust shape if needed
        print(f'This digit is {np.argmax(prediction)}')

    except Exception as e:
        print(f'Error processing image {image_number}: {e}')

    finally:
        image_number += 1  # Increment the image number

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
This digit is 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
This digit is 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
This digit is 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
This digit is 7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
This digit is 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
This digit is 8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
This digit is 9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
This digit is 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
This digit is 9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
This digit is 6
No more images found.
